<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module_1/images/SN_web_lightmode.png" width="300"> 
</center>


<h1>Data Wrangling with dplyr</h1>

Estimated time needed: **60** minutes


## Lab Overview:

In this lab, you will focus on wrangling the Seoul bike-sharing demand historical dataset. This is the core dataset to build a predictive model later.

It contains the following columns:

- `DATE` : Year-month-day
- `RENTED BIKE COUNT`- Count of bikes rented at each hour
- `HOUR`- Hour of he day
- `TEMPERATURE` - Temperature in Celsius
- `HUMIDITY` - Unit is `%`
- `WINDSPEED` - Unit is `m/s`
- `VISIBILITY` - Multiplied by 10m
- `DEW POINT TEMERATURE` - The temperature to which the air would have to cool down in order to reach saturation, unit is Celsius
- `SOLAR RADIATION` - MJ/m2
- `RAINFALL` - mm
- `SNOWFALL` - cm
- `SEASONS` - Winter, Spring, Summer, Autumn
- `HOLIDAY` - Holiday/No holiday
- `FUNCTIONAL DAY` - NoFunc(Non Functional Hours), Fun(Functional hours)


For this dataset, you will be asked to use `tidyverse` to perform the following data wrangling tasks:

- **TASK: Detect and handle missing values**
- **TASK: Create indicator (dummy) variables for categorical variables**
- **TASK: Normalize data**

Let's start!


First import the necessary library for this data wrangling task:


In [1]:
# Check if you need to install the `tidyverse` library
# require("tidyverse")
library(tidyverse)
library(stringr)  # For string manipulation
library(readr)    # For reading CSV files (read_csv())
library(dplyr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Then load the bike-sharing system data from the csv processed in the previous lab:


In [2]:
bike_sharing_df <- read_csv("raw_seoul_bike_sharing.csv")

Rows: 8760 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): DATE, SEASONS, HOLIDAY, FUNCTIONING_DAY
dbl (10): RENTED_BIKE_COUNT, HOUR, TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBI...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
# Or you may read it from here again
# url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_seoul_bike_sharing.csv"
# Notice some column names in the raw datasets are not standalized if you haven't done them properly in the previous lab

First take a quick look at the dataset:


In [4]:
summary(bike_sharing_df)
dim(bike_sharing_df)

     DATE           RENTED_BIKE_COUNT      HOUR        TEMPERATURE    
 Length:8760        Min.   :   2.0    Min.   : 0.00   Min.   :-17.80  
 Class :character   1st Qu.: 214.0    1st Qu.: 5.75   1st Qu.:  3.40  
 Mode  :character   Median : 542.0    Median :11.50   Median : 13.70  
                    Mean   : 729.2    Mean   :11.50   Mean   : 12.87  
                    3rd Qu.:1084.0    3rd Qu.:17.25   3rd Qu.: 22.50  
                    Max.   :3556.0    Max.   :23.00   Max.   : 39.40  
                    NA's   :295                       NA's   :11      
    HUMIDITY       WIND_SPEED      VISIBILITY   DEW_POINT_TEMPERATURE
 Min.   : 0.00   Min.   :0.000   Min.   :  27   Min.   :-30.600      
 1st Qu.:42.00   1st Qu.:0.900   1st Qu.: 940   1st Qu.: -4.700      
 Median :57.00   Median :1.500   Median :1698   Median :  5.100      
 Mean   :58.23   Mean   :1.725   Mean   :1437   Mean   :  4.074      
 3rd Qu.:74.00   3rd Qu.:2.300   3rd Qu.:2000   3rd Qu.: 14.800      
 Max.   :98.

[1] 8760   14

From the summary, we can observe that:

Columns `RENTED_BIKE_COUNT`, `TEMPERATURE`, `HUMIDITY`, `WIND_SPEED`, `VISIBILITY`, `DEW_POINT_TEMPERATURE`, `SOLAR_RADIATION`, `RAINFALL`, `SNOWFALL` are numerical variables/columns and require normalization. Moreover, `RENTED_BIKE_COUNT` and `TEMPERATURE` have some missing values (NA's) that need to be handled properly.

`SEASONS`, `HOLIDAY`, `FUNCTIONING_DAY` are categorical variables which need to be converted into indicator columns or dummy variables.
Also, `HOUR` is read as a numerical variable but it is in fact a categorical variable with levels ranging from 0 to 23.

Now that you have some basic ideas about how to process this bike-sharing demand dataset, let's start working on it!


# TASK: Detect and handle missing values


 The `RENTED_BIKE_COUNT` column has about 295 missing values, and `TEMPERATURE` has about 11 missing values. Those missing values could be caused by not being recorded, or from malfunctioning bike-sharing systems or weather sensor networks. In any cases, the identified missing values have to be properly handled.


Let's first handle missing values in `RENTED_BIKE_COUNT` column:


Considering `RENTED_BIKE_COUNT` is the response variable/dependent variable, i.e., we want to predict the `RENTED_BIKE_COUNT` using other predictor/independent variables later, and we normally can not allow missing values for the response variable, so missing values for response variable must be either dropped or imputed properly. 

We can see that `RENTED_BIKE_COUNT` only has about 3% missing values (295 / 8760). As such, you can safely drop any rows whose `RENTED_BIKE_COUNT` has missing values.


_TODO:_ Drop rows with missing values in the `RENTED_BIKE_COUNT` column


In [5]:
# Drop rows with `RENTED_BIKE_COUNT` column == NA

# Check the number of rows before dropping
rows_before <- nrow(bike_sharing_df)

# Drop rows with missing values in RENTED_BIKE_COUNT
bike_sharing_df <- bike_sharing_df %>%
  filter(!is.na(RENTED_BIKE_COUNT))

# Check the number of rows after dropping
rows_after <- nrow(bike_sharing_df)

# Print the results
cat("Number of rows before dropping:", rows_before, "\n")
cat("Number of rows after dropping:", rows_after, "\n")

Number of rows before dropping: 8760 
Number of rows after dropping: 8465 


In [6]:
# Print the dataset dimension again after those rows are dropped

# Print the dataset dimensions
cat("Dataset dimensions after dropping rows with missing RENTED_BIKE_COUNT:\n")
print(dim(bike_sharing_df))

# Optionally, you can also print a more detailed summary
cat("\nDataset summary after dropping rows:\n")
print(summary(bike_sharing_df))

#JUST IN CASE, let's double check!
unique(bike_sharing_df$RENTED_BIKE_COUNT)

##so, there's no NA in RENTED_BIKE_COUNT !

Dataset dimensions after dropping rows with missing RENTED_BIKE_COUNT:
[1] 8465   14

Dataset summary after dropping rows:
     DATE           RENTED_BIKE_COUNT      HOUR        TEMPERATURE    
 Length:8465        Min.   :   2.0    Min.   : 0.00   Min.   :-17.80  
 Class :character   1st Qu.: 214.0    1st Qu.: 6.00   1st Qu.:  3.00  
 Mode  :character   Median : 542.0    Median :12.00   Median : 13.40  
                    Mean   : 729.2    Mean   :11.51   Mean   : 12.75  
                    3rd Qu.:1084.0    3rd Qu.:18.00   3rd Qu.: 22.60  
                    Max.   :3556.0    Max.   :23.00   Max.   : 39.40  
                                                      NA's   :11      
    HUMIDITY       WIND_SPEED      VISIBILITY   DEW_POINT_TEMPERATURE
 Min.   : 0.00   Min.   :0.000   Min.   :  27   Min.   :-30.600      
 1st Qu.:42.00   1st Qu.:0.900   1st Qu.: 935   1st Qu.: -5.100      
 Median :57.00   Median :1.500   Median :1690   Median :  4.700      
 Mean   :58.15   Mean   :1.72

[1]  254  204  173  107   78  100  181  460  930  490  339  360  449  451
  [15]  447  463  484  555  862  600  426  405  398  323  328  308  262  167
  [29]   89   79   70  146  219  334  388  479  606  611  589  618  572  489
  [43]  385  359  377  354  366  342  227  228  117   76   32   13   22   85
  [57]  152  200  273  333  495  563  621  598  545  443  422  397  350  303
  [71]  285  186  112   65   41   82  498  937  509  311  379  393  391  338
  [85]  341  375  515  807  568  353  248  216  198  124   62   42  172  381
  [99]  812  462  269  321  325  435  529  789  597  418  415  244  145  144
 [113]   58   87  142  802  440  307  442  416  412  496  239  205  190  165
 [127]  222  193  218  149  102   83  137  367  754  431  394  390  430  471
 [141]  552  795  507  402  389  259  233  230  131   99   71   84  158  387
 [155]  780  395  261  310  355  362  401  500  752  477  384  336  347  275
 [169]  267  296  237  169   96   56  127  291  351  427  486  569  546  481
 [183]  298  289  265  326  280  243    9   20   35   31   19  119  182  235
 [197]  180  214  226  210  148  125  111   67   45   44   86  136  653  212
 [211]  247  241  253  277  361  557  371  266  174  133   95   54   46  104
 [225]  580  184  201  225  258  250  305  260  270  256  168  183  120   68
 [239]  109  591  340  202  251  240  278  392  537  373  263  271  171  170
 [253]   80   52  108  279  189  282  294  301  335  382  612  410  316  309
 [267]  211  162  110   69  129  658  330  221  329  464  682  320  300  217
 [281]  302   66   90  195  252  292  143  159  155  175  132   39   53   94
 [295]  122  223  268  157  166  138  151   97   38   29   27   36  116  281
 [309]  160  128   93   55  199  191  178  276  224  213  466  209  197  153
 [323]   49  423  290  255  531  288  196  161   64  114  283  544  386  671
 [337]  314  249  337  312   59   63  188  473  453   12    3    4    7   28
 [351]  115   72   61  179  274  319  272  185  176  578  293  246  317  220
 [365]  156   81  105  502  231  315  242  164  163   50  103  257  554  286
 [379]  407  332  135  636  363  370  419  504  534  574  236  234  192   88
 [393]  208  356  345   40   30   43   25  215  206  121  139  344  203  154
 [407]  655  295  675  403  364  304  130  670  297  299  637  150  123   57
 [421]  646  264  284  603  417  306  118   47   75  327  665  232  343  446
 [435]  635  322   48  245  352  207   60  331  368  349   77  374  701  318
 [449]  126  558  177  437   98   33  187   92  140  106  134   74  444  365
 [463]  455  141   51  650  376  592  409  380  501  313  506  648  468  702
 [477]  101  406   73  396  383  683   18   24   34  113  147   21   26  450
 [491]  438  491  238  229   37  459  194  542   91   23  433  469  448  564
 [505]  378  659  517  475  604  461  559   14  372  404  522  819  533  357
 [519]  727  514  764  358  718  784  783  465  730  499  348  346  470  669
 [533]  421  454  467  482  688  786  428  521  929  619  483  512  767  411
 [547]  897  616  816   17    8   11   10  458  617  631  854  414  605  771
 [561] 1025 1238 1285 1197  920  630  588  516   15  429  492  436    2  550
 [575]  865 1155  782  583  997  556  528  567  680  883 1246  833  666  625
 [589] 1032  566  441  505  620  849 1365  799  587  452  594  757 1091  586
 [603] 1041  570  510  686  834 1231  535  497  906  931  972  829  664  573
 [617]  425  843 1154 1294 1205 1028  776  596  530  662 1066  581  694 1425
 [631]  946  801  695  668 1215  571  626  678  705  815  890 1085 1749 1224
 [645]  986  982  912 1371  761  643  813  878 1049 1325 2100 1478 1275 1181
 [659]  983  676  287  652 1176  715  772  923  976 1160 1641  974  774  710
 [673]  660  520  561  742  933 1111 1514 1440 1272  934  624  562  751  881
 [687]  993 1128 1059 1200  818  711  731  565  677 1212  661  488  478  518
 [701]  719  716  543  494 1097  614  615 1174  674  758  999 1495  739  726
 [715] 1216  584  690  901 1108 1611  932  724  638  424  456  806  880  855

Now that you have handled  missing values in the `RENTED_BIKE_COUNT` variable, let's continue processing missing values for the `TEMPERATURE` column.


Unlike the `RENTED_BIKE_COUNT` variable, `TEMPERATURE` is not a response variable. However, it is still an important predictor variable - as you could imagine, there may be a positve correlation between `TEMPERATURE` and `RENTED_BIKE_COUNT`. For example, in winter time with lower temperatures, people may not want to ride a bike, while in summer with nicer weather, they are more likely to rent a bike.


How do we handle missing values for `TEMPERATURE`? We could simply remove the rows but it's better to impute them because `TEMPERATURE` should be relatively easy and reliable to estimate statistically.


Let's first take a look at the missing values in the TEMPERATURE column.


In [7]:
# Filter rows with missing values in TEMPERATURE
missing_temp_rows <- bike_sharing_df %>%
  filter(is.na(TEMPERATURE))

# Display rows with missing TEMPERATURE values
print(missing_temp_rows)

# Count the number of missing values in TEMPERATURE
num_missing_temp <- sum(is.na(bike_sharing_df$TEMPERATURE))
cat("Number of missing values in TEMPERATURE:", num_missing_temp, "\n")

# Check if all missing TEMPERATURE values occur in a specific season (e.g., Summer)
missing_seasons <- missing_temp_rows %>%
  group_by(SEASONS) %>%
  summarize(count = n())

print(missing_seasons)

# A tibble: 11 × 14
   DATE       RENTED_BIKE_COUNT  HOUR TEMPERATURE HUMIDITY WIND_SPEED VISIBILITY
   <chr>                  <dbl> <dbl>       <dbl>    <dbl>      <dbl>      <dbl>
 1 07/06/2018              3221    18          NA       57        2.7       1217
 2 12/06/2018              1246    14          NA       45        2.2       1961
 3 13/06/2018              2664    17          NA       57        3.3        919
 4 17/06/2018              2330    17          NA       58        3.3        865
 5 20/06/2018              2741    19          NA       61        2.7       1236
 6 30/06/2018              1144    13          NA       87        1.7        390
 7 05/07/2018               827    10          NA       75        1.1       1028
 8 11/07/2018               634     9          NA       96        0.6        450
 9 12/07/2018               593     6          NA       93        1.1        852
10 21/07/2018               347     4          NA       77        1.2       1203
11 21/08

It seems that all of the missing values for `TEMPERATURE` are found in `SEASONS == Summer`, so it is reasonable to impute those missing values with the summer average temperature.


_TODO:_ Impute missing values for the TEMPERATURE column using its mean value.


In [8]:
# Calculate the summer average temperature
mean_temperature <- mean(bike_sharing_df$TEMPERATURE, na.rm = TRUE)

print(mean_temperature)

[1] 12.75432


In [9]:
# Impute missing values for TEMPERATURE column with summer average temperature
bike_sharing_df <- bike_sharing_df %>%
  mutate(TEMPERATURE = ifelse(is.na(TEMPERATURE), mean_temperature, TEMPERATURE))

In [10]:
# Print the summary of the dataset again to make sure no missing values in all columns
summary(bike_sharing_df)

     DATE           RENTED_BIKE_COUNT      HOUR        TEMPERATURE    
 Length:8465        Min.   :   2.0    Min.   : 0.00   Min.   :-17.80  
 Class :character   1st Qu.: 214.0    1st Qu.: 6.00   1st Qu.:  3.00  
 Mode  :character   Median : 542.0    Median :12.00   Median : 13.40  
                    Mean   : 729.2    Mean   :11.51   Mean   : 12.75  
                    3rd Qu.:1084.0    3rd Qu.:18.00   3rd Qu.: 22.60  
                    Max.   :3556.0    Max.   :23.00   Max.   : 39.40  
    HUMIDITY       WIND_SPEED      VISIBILITY   DEW_POINT_TEMPERATURE
 Min.   : 0.00   Min.   :0.000   Min.   :  27   Min.   :-30.600      
 1st Qu.:42.00   1st Qu.:0.900   1st Qu.: 935   1st Qu.: -5.100      
 Median :57.00   Median :1.500   Median :1690   Median :  4.700      
 Mean   :58.15   Mean   :1.726   Mean   :1434   Mean   :  3.945      
 3rd Qu.:74.00   3rd Qu.:2.300   3rd Qu.:2000   3rd Qu.: 15.200      
 Max.   :98.00   Max.   :7.400   Max.   :2000   Max.   : 27.200      
 SOLAR_RADIAT

In [11]:
# Save the dataset as `seoul_bike_sharing.csv`
write_csv(bike_sharing_df, "seoul_bike_sharing.csv")

# TASK: Create indicator (dummy) variables for categorical variables


Regression models can not process categorical variables directly, thus we need to convert them into indicator variables. 


In the bike-sharing demand dataset, `SEASONS`, `HOLIDAY`, `FUNCTIONING_DAY` are categorical variables. 
Also, `HOUR` is read as a numerical variable but it is in fact a categorical variable with levels ranged from 0 to 23.


_TODO:_ Convert `HOUR` column from numeric into character first:


In [12]:
# Using mutate() function to convert HOUR column into character type

# Convert HOUR column from numeric to character
bike_sharing_df <- bike_sharing_df %>%
  mutate(HOUR = as.character(HOUR))

# Verify the conversion
str(bike_sharing_df$HOUR)  # Check the structure of the HOUR column
class(bike_sharing_df$HOUR)  # Confirm that it is now a character column

 chr [1:8465] "0" "1" "2" "3" "4" "5" "6" "7" "8" "9" "10" "11" "12" "13" ...


[1] "character"

`SEASONS`, `HOLIDAY`, `FUNCTIONING_DAY`,  `HOUR` are all character columns now and are ready to be converted into indicator variables.

For example, `SEASONS` has four categorical values: `Spring`, `Summer`, `Autumn`, `Winter`. We thus need to create four indicator/dummy variables `Spring`, `Summer`, `Autumn`, and `Winter` which only have the value 0 or 1.

So, given a data entry with the value `Spring` in the `SEASONS` column, the values for the four new columns `Spring`, `Summer`, `Autumn`, and `Winter` will be set to 1 for `Spring` and 0 for the others:

|Spring|Summer|Autumn|Winter|
|----- |------|------|------|
|     1|     0|     0|     0| 


_TODO:_ Convert `SEASONS`, `HOLIDAY`, `FUNCTIONING_DAY`, and `HOUR` columns into indicator columns.

Note that if `FUNCTIONING_DAY` only contains one categorical value after missing values removal, then you don't need to convert it to an indicator column.


In [13]:
# Convert SEASONS, HOLIDAY, FUNCTIONING_DAY, and HOUR columns into indicator columns.

# Check if FUNCTIONING_DAY has only one categorical value after missing value removal
unique_functioning_day <- unique(bike_sharing_df$FUNCTIONING_DAY)
if (length(unique_functioning_day) == 1) {
  cat("FUNCTIONING_DAY has only one unique value:", unique_functioning_day, "\n")
  # Remove FUNCTIONING_DAY since it does not need to be converted
  bike_sharing_df <- subset(bike_sharing_df, select = -FUNCTIONING_DAY)
}

# Convert SEASONS, HOLIDAY, and HOUR into factors
bike_sharing_df$SEASONS <- as.factor(bike_sharing_df$SEASONS)
bike_sharing_df$HOLIDAY <- as.factor(bike_sharing_df$HOLIDAY)
bike_sharing_df$HOUR <- as.factor(bike_sharing_df$HOUR)

# Create dummy variables for SEASONS
seasons_dummies <- model.matrix(~ SEASONS - 1, data = bike_sharing_df)

# Create dummy variables for HOLIDAY
holiday_dummies <- model.matrix(~ HOLIDAY - 1, data = bike_sharing_df)

# Create dummy variables for HOUR
hour_dummies <- model.matrix(~ HOUR - 1, data = bike_sharing_df)

# Combine the original dataset with the dummy variables and remove original columns
bike_sharing_df <- cbind(
  bike_sharing_df[, !(names(bike_sharing_df) %in% c("SEASONS", "HOLIDAY", "HOUR"))],
  seasons_dummies,
  holiday_dummies,
  hour_dummies
)

ERROR: Error in parse(text = input): <text>:12:63: unexpected symbol
11: # Convert SEASONS, HOLIDAY, and HOUR into factors
12: bike_sharing_df$SEASONS <- as.factor(bike_sharing_df$SEASONS) bike_sharing_df
                                                                  ^


In [ ]:
# Print the dataset summary again to make sure the indicator columns are created properly
summary(bike_sharing_df)

In [ ]:
# Save the dataset as `seoul_bike_sharing_converted.csv`
# write_csv(dataframe, "seoul_bike_sharing_converted.csv")
write_csv(bike_sharing_df, "seoul_bike_sharing_converted.csv")

# TASK: Normalize data


Columns `RENTED_BIKE_COUNT`, `TEMPERATURE`, `HUMIDITY`, `WIND_SPEED`, `VISIBILITY`, `DEW_POINT_TEMPERATURE`, `SOLAR_RADIATION`, `RAINFALL`, `SNOWFALL` are numerical variables/columns with different value units and range. Columns with large values may adversely influence (bias) the predictive models and degrade model accuracy. Thus, we need to perform normalization on these numeric columns to transfer them into a similar range.


In this project, you are asked to use Min-max normalization: 

**Min-max** rescales each value in a column by first subtracting the minimum value of the column from each value, and then divides the result by the difference between the maximum and minimum values of the column. So the column gets re-scaled such that the minimum becomes 0 and the maximum becomes 1.

$$x_{new} = \frac{x_{old} - x_{min}}{x_{max} - x_{min}}$$


_TODO:_ Apply min-max normalization on `RENTED_BIKE_COUNT`, `TEMPERATURE`, `HUMIDITY`, `WIND_SPEED`, `VISIBILITY`, `DEW_POINT_TEMPERATURE`, `SOLAR_RADIATION`, `RAINFALL`, `SNOWFALL`


In [ ]:
# Use the `mutate()` function to apply min-max normalization on columns 
# `RENTED_BIKE_COUNT`, `TEMPERATURE`, `HUMIDITY`, `WIND_SPEED`, `VISIBILITY`, `DEW_POINT_TEMPERATURE`, `SOLAR_RADIATION`, `RAINFALL`, `SNOWFALL`
# Load necessary libraries
library(readr)

# Load the dataset
bike_sharing_df <- read_csv("seoul_bike_sharing_converted.csv")

# List of columns to normalize
columns_to_normalize <- c("RENTED_BIKE_COUNT", "TEMPERATURE", "HUMIDITY", 
                          "WIND_SPEED", "VISIBILITY", "DEW_POINT_TEMPERATURE", 
                          "SOLAR_RADIATION", "RAINFALL", "SNOWFALL")

# Apply Min-Max normalization manually for each column
for (col in columns_to_normalize) {
  bike_sharing_df[[col]] <- (bike_sharing_df[[col]] - min(bike_sharing_df[[col]], na.rm = TRUE)) / 
                            (max(bike_sharing_df[[col]], na.rm = TRUE) - min(bike_sharing_df[[col]], na.rm = TRUE))
}

In [ ]:
# Print the summary of the dataset again to make sure the numeric columns range between 0 and 1
summary(bike_sharing_df)

In [ ]:
# Save the dataset as `seoul_bike_sharing_converted_normalized.csv`
# write_csv(dataframe, "seoul_bike_sharing_converted_normalized.csv")
write_csv(bike_sharing_df, "seoul_bike_sharing_converted_normalized.csv")

## Standardize the column names again for the new datasets


Since you have added many new indicator variables, you need to standardize their column names again by using the following code:


In [ ]:
# Dataset list
dataset_list <- c('seoul_bike_sharing.csv', 'seoul_bike_sharing_converted.csv', 'seoul_bike_sharing_converted_normalized.csv')

for (dataset_name in dataset_list){
    # Read dataset
    dataset <- read_csv(dataset_name)
    # Standardized its columns:
    # Convert all columns names to uppercase
    names(dataset) <- toupper(names(dataset))
    # Replace any white space separators by underscore, using str_replace_all function
    names(dataset) <- str_replace_all(names(dataset), " ", "_")
    # Save the dataset back
    write.csv(dataset, dataset_name, row.names=FALSE)
}

# Next Steps


Great! Now that you have processed all of the necessary datasets, you are ready to perform exploratory data analysis to get some inital insights from them.


## Authors

<a href="https://www.linkedin.com/in/yan-luo-96288783/" target="_blank">Yan Luo</a>


### Other Contributors

Jeff Grossman


<!--## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description           |
| ----------------- | ------- | ---------- | ---------------------------- |
| 2021-04-08        | 1.0     | Yan        | Initial version created      |
|                   |         |            |                              |
|                   |         |            |                              |-->

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
